## Linear congruential generator

http://www.algorytm.org/liczby-pseudolosowe/generator-lcg-liniowy-generator-kongruentny.html \
https://rosettacode.org/wiki/Linear_congruential_generator 

x<sub>i</sub>=(ax<sub>i-1</sub>-c) mod m

Borland C/C++ initial values:
* a=22695477 
* m=2^32 
* c=1 

In [101]:
a,c,x,m=19,1,0,381 #really bad initial conditions
#a,c,x,m=22695477,1,42,2**32 #too good
#a,c,x,m=3,1,42,2**32 #random "a" by fair dice roll

def get_prnd():
    global x
    x=(a*x+c)%m
    yield x

In [102]:
next(get_prnd()) #953210035

1

In [103]:
list(get_pseudorandom(10))

[20, 0, 1, 20, 0, 1, 20, 0, 1, 20]

In [104]:
def get_pseudorandom(N=1):
    return (next(get_prnd()) for _ in range(N)) if N>1 else next(get_prnd())

In [105]:
import subprocess
import sys

def acc_FIPS(lst):
    out,_=subprocess.Popen(["rngtest"],
                 stdin=subprocess.PIPE,
                 stdout=subprocess.PIPE,
                 stderr=subprocess.STDOUT).communicate(b"".join(map(lambda x: x.to_bytes(4,'little'),lst)))
    out=out.decode('utf-8')
    print(out)
    succ,fail=[int(line.split()[-1]) for line in out.splitlines()[7:9]]
    return succ/(succ+fail) if (succ+fail)>0 else 0

In [106]:
%%time
acc_FIPS(get_pseudorandom(10**7))

rngtest 5
Copyright (c) 2004 by Henrique de Moraes Holschuh
This is free software; see the source for copying conditions.  There is NO warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

rngtest: starting FIPS tests...
rngtest: entropy source drained
rngtest: bits received from input: 320000000
rngtest: FIPS 140-2 successes: 0
rngtest: FIPS 140-2 failures: 15999
rngtest: FIPS 140-2(2001-10-10) Monobit: 15999
rngtest: FIPS 140-2(2001-10-10) Poker: 15999
rngtest: FIPS 140-2(2001-10-10) Runs: 15999
rngtest: FIPS 140-2(2001-10-10) Long run: 15999
rngtest: FIPS 140-2(2001-10-10) Continuous run: 0
rngtest: input channel speed: (min=1818181818.182; avg=15504409341.991; max=0.000)bits/s
rngtest: FIPS tests speed: (min=60.169; avg=221.434; max=241.437)Mibits/s
rngtest: Program run time: 8185484 microseconds

CPU times: user 6.58 s, sys: 312 ms, total: 6.89 s
Wall time: 8.19 s


0.0

## Improvements

In [107]:
from glob import iglob 
import gzip
import os

pathgz_iter = ( path for path in iglob('/home/**',recursive=True) 
                  if len(path)>3 and path[-3:]=='.gz' and os.path.getsize(path)>102400 )

In [108]:
def get_compressed_32bits():
    for path in pathgz_iter:
        print(path,os.path.getsize(path))
        with open(path,'rb') as f:
            #headers and stuff
            #TODO:check later exact size
            #from this http://www.onicos.com/staff/iz/formats/gzip.html
            f.read(100)
            #payload
            while True:
                seed=f.read(4)
                if not seed:
                    break
                yield int.from_bytes(seed,byteorder='little')

In [109]:
def get_improved(N=10):
    for i,vals in enumerate(zip(get_pseudorandom(N),get_compressed_32bits())):
        if i==N:
            break
        pseudo,seed = vals
        yield pseudo ^ seed

In [110]:
%%time
acc_FIPS(get_improved(10**7))

/home/fisher/projects/repo/master/mosesdecoder/scripts/tests/cs-en-sample/lm.en.gz 237898
/home/fisher/projects/repo/dev/cmake_update/cmake-3.16.4.tar.gz 9113021
/home/fisher/projects/wykop/mongo_dump/dump/wykop/post.bson.gz 918184403
rngtest 5
Copyright (c) 2004 by Henrique de Moraes Holschuh
This is free software; see the source for copying conditions.  There is NO warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

rngtest: starting FIPS tests...
rngtest: entropy source drained
rngtest: bits received from input: 320000000
rngtest: FIPS 140-2 successes: 11590
rngtest: FIPS 140-2 failures: 4409
rngtest: FIPS 140-2(2001-10-10) Monobit: 2965
rngtest: FIPS 140-2(2001-10-10) Poker: 2822
rngtest: FIPS 140-2(2001-10-10) Runs: 2801
rngtest: FIPS 140-2(2001-10-10) Long run: 7
rngtest: FIPS 140-2(2001-10-10) Continuous run: 6
rngtest: input channel speed: (min=2222222222.222; avg=15206729398.346; max=0.000)bits/s
rngtest: FIPS tests speed: (min=34.060; avg=90.531; max=

0.7244202762672667

## Conclusions

TODO:check if **a** needs to be prime

* for Borland C/C++ initial values we get simmilar accuracy as /dev/random 99.9%
* bad initial conditions we get LCG with 0% accuracy on FIPS and can't get to /dev/random level with our improvements (max<sub>acc</sub>=70%)
* transition case **a** = 3 and **m** = 2^32 still below 1% acc on FIPS, however after our improvements we get 99.9% acc